In [ ]:
from google.colab import drive
drive.mount('/content/drive')
CURR_DIR = "/content/drive/My Drive"
DATA_FILE = "profiles.csv"

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import os
import nltk
import sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from scipy import sparse
np.set_printoptions(threshold=1000)

In [ ]:
profiles = pd.read_csv(os.path.join(CURR_DIR, DATA_FILE))
print(profiles)

       age  ...     status
0       22  ...     single
1       35  ...     single
2       38  ...  available
3       23  ...     single
4       29  ...     single
...    ...  ...        ...
59941   59  ...     single
59942   24  ...     single
59943   42  ...     single
59944   27  ...     single
59945   39  ...     single

[59946 rows x 31 columns]


In [ ]:
train_df = profiles.sample(10000, random_state=42)

In [ ]:
def parseEssay(df):
  df['all']=''
  essays=['essay0','essay1','essay2','essay3','essay4','essay5','essay6','essay7','essay8','essay9']
  for i in essays[0:2]:
    df[i]=df[i].replace(np.nan,'')
    df['all']=df[i]+''+df['all']
  essay_list = df['essay5'].to_list()
  #essay_list.append( df['essay1'].to_list())

  # parses out the NaN's and leaves them empty
  print("parsing out nans")
  for i in range(len(essay_list)):
    if not isinstance(essay_list[i], str):
      essay_list[i] = ""

  return essay_list

def createWordVectors(train_essays):
  
  vectorizer = CountVectorizer()
  vectorizer.fit(train_essays)

  word_vecs = vectorizer.transform(train_essays)

  return word_vecs, vectorizer

train_essays = parseEssay(train_df)
word_vecs, vectorizer = createWordVectors(train_essays)
print(len(train_essays))
X = word_vecs.toarray()
y=(train_df.sex=='m').values.astype(np.int)
# for printing out the actual vocab encoding vec in case you want it
# print(vectorizer.vocabulary_)
# encoding document
print(X.shape)
print(y.shape)
print(len(vectorizer.vocabulary_))

parsing out nans
10000
(10000, 13570)
(10000,)
13570


In [ ]:
kf=KFold(n_splits=10)
kf.get_n_splits(X)
tracc=[]
teacc=[]
t=0.01
for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    #X_Train=X_train.sample(frac=t,replace=False,random_state=32)
    y_train, y_test = y[train_index], y[test_index]
    #y_Train=y_train.sample(frac=t,replace=False,random_state=32)

#xtrain, xtest, ytrain, ytest = train_test_split(X, y)
    acc = MultinomialNB(alpha=0.1).fit(X_train, y_train)
    t=acc.score(X_train, y_train)
    st=acc.score(X_test,y_test)
    tracc.append(t)
    teacc.append(st)

print(np.mean(tracc),np.mean(teacc))

0.7529333333333333 0.5974
